<a href="https://colab.research.google.com/github/pinballsurgeon/NLP_Engine_Study/blob/main/LLM_Alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: Enable CUDA GPUs in runtime settings 

Install

In [3]:
# install transformers
!pip install transformers -q

# install openai
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import

In [4]:
# standard dan-dard
import os
import io
import re
import numpy
import pandas
import string
import seaborn

# pytorch
import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# tensorflow
import tensorflow_hub
import tensorflow as tf
from tensorboard.plugins import projector
from tensorflow.keras.layers import TextVectorization
 
# universal encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = tensorflow_hub.load(module_url)

# set up a logs directory, so Tensorboard knows where to look for files.
log_dir='/logs/embedding_projector/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# LM transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed

# gpt3 
import openai

# supply openai api key via file 
openai.api_key = open('openai_key').read()

# load tensorboar dextension
%load_ext tensorboard

Download

In [5]:
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b3", use_cache=True)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b3")

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.21G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Helper functions

In [6]:
### 5. send openai question, receive response
def get_openai_response(question):

    # format open au request
    response = openai.Completion.create(
                      engine="text-davinci-001",
                      prompt=question,
                      temperature=.5,
                      max_tokens=250,
                      top_p=1,
                      frequency_penalty=50,
                      presence_penalty=0 )

    # parse and process open ai response
    response_choices = response["choices"]

    # replace blanks
    response = response_choices[0]["text"].strip('\n')

    return response

In [11]:
def get_openai_generated_topics(topic, obj, state):
  
  new_tokens_list = []

  responses = []

  # retrieve 
  # retrieval = get_openai_response('from the perspetive of a %s resident, list several single word %s related to %s' % ( state , obj , topic ))
  # retrieval = get_openai_response('list one word %s related to %s' % (obj , topic))
  retrieval = get_openai_response('what is today')
  
  responses.append(retrieval)

  new_tokens = re.split("; |, |\*|\n",responses[0].lower())

  # return new_tokens
  return responses

In [12]:
get_openai_generated_topics('g','f','g')

["'s date\n\n\nThe current day is October 3rd."]

In [13]:
prompt = f'what is today'

In [14]:
input_ids = tokenizer(prompt, return_tensors="pt").to(0)

In [19]:
sample = model.generate(**input_ids, num_beams = 2, num_beam_groups = 2, top_k=1, temperature=0.9, repetition_penalty = 2.0, diversity_penalty=2.0)

/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


In [18]:
print(tokenizer.decode(sample[0], truncate_before_pattern=[r"\n\n^#", "^'''","\n\n\n"]))

what is today known as the “New York Times”, and it was founded in 1851 by William
